# 20-Tableau - CitiBike

@Author Jeffery Brown (daddyjab)<br>
@Date 4/14/19<br>
@File Citibike_Exploration_JAB

# Exploratory Analysis and Data Munging

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import datetime as dt

from pprint import pprint


In [2]:
# Functions to find the distance between 2 lat/long coordinates
# Description: https://pypi.org/project/geopy
from geopy.distance import (distance, great_circle)

In [3]:
def gen_paths_df( a_df ):
# Function to process a DataFrame containing CitiBike data
# to provide paths in a format usable by Tableau
#
# Arguments:
#    a_df: DataFrame populated with CitiBike data
#
# Returns:
#    retval: DataFrame with bike path populated as needed by Tableau

    # If no DataFrame has been provided, then return
    if a_df is None:
        return None
    
    # Sorting by Bike ID then Start time then Stop Time to ensure that
    # the movement of each bike from station to station is in order
    a_df.sort_values(by=['bikeid','starttime', 'stoptime'], inplace=True)
    
    # Reset the indices
    a_df.reset_index(drop=True, inplace=True)

    # Loop through each record and build a path per bike with
    # Unique pathid for every path, and an ordered list of stations on the path
    # Also, include all of the related data items with each point on the path

    cb_dict_list = []
    bikeid_last = None;
    pathid_last = None;
    endstationid_last = None;

    for r in a_df.itertuples():

        # Get the Start Time as a Timestamp object (for later)
        start_time = pd.to_datetime(r.starttime)
        
        # Get the Station IDs and ensure they are integers
        try:
            start_station_id = int(r._4)
            end_station_id = int(r._8)
        except ValueError:
            # If an error occurs with Start or End Station ID
            # Just skip this entry since a Path ID cannot be generated
            print(f">> *** ValueError: start_station_id: {r._4}, end_station_id: {r._8} ")
            continue

        # If the Bike ID has changed or
        # if the current Start Station ID is different than
        # the previous End Station ID, then a new path has started

        # print(f">> Processing Bike ID: {r.bikeid}, Start Station ID: {start_station_id} vs. Last End Station ID: {endstationid_last}")
        if r.bikeid != bikeid_last or start_station_id != endstationid_last:
            
            # Create a new unique Path ID using:
            # Bike ID, Start Date and Time (YYYY-MM-DD-hh:mm:ss), and Start Station ID where this path started
            pathid_last = "_".join([ str(r.bikeid), start_time.strftime("%Y-%m-%d_%H_%M_%S"), str(start_station_id) ])
            print(f">>>> Starting a new Path ID: {pathid_last}")

            # Start the Path Order at 1
            path_order = 1

        # Calculate the distance between the start and end stations
        start_station_coord = ( r._6, r._7 )
        end_station_coord = ( r._10, r._11 )
        
        try:
            trip_distance = distance( start_station_coord, end_station_coord ).miles
        except:
            trip_distance = None

        # Calculate the Trip Duration in minutes (vs. seconds)
        try:
            trip_duration_min = r.tripduration / 60.0
            
        except:
            trip_duration_min = None

        # Calculate the Trip Speed in mph
        try:
            trip_speed_mph = trip_distance / (trip_duration_min / 60.0)

        except:
            trip_speed_mph = None

        # Clarify the User Type
        try:
            if r.usertype == "Subscriber":
                user_type_adj = "Annual Subscriber"
            else:
                user_type_adj = "24h or 3d Pass User"
        except:
            user_type_adj = None
                
        # Clarify the Gender
        try:
            if r.gender == 1:
                user_gender_adj = "Male"
            elif r.gender == 2:
                user_gender_adj = "Female"
            else:
                user_gender_adj = "Unknown"
        except:
            user_gender_adj = None
                
        # Calculate Age
        try:
            # If the Birth Year is less than 1900,
            # then assume it is invalid data
            if r._14 > 1900:
                user_age = start_time.year - r._14
            else:
                user_age = None
                
        except:
            user_age = None            

        # Build a dictionary object to be added to the list
        cb_path_item = {
            'Bike ID' : int(r.bikeid),
            'Path ID': pathid_last,
            'Path Order': path_order,
            'Start Time' : r.starttime,
            'Stop Time' : r.stoptime,
            'Trip Duration (mins)' : trip_duration_min,
            'Trip Distance (miles)' : trip_distance,
            'Trip Speed (mph)' : trip_speed_mph,
            'Start Station ID' : start_station_id,
            'Start Station Name' : r._5,
            'End Station ID' : end_station_id,
            'End Station Name' : r._9,
            'Start Station Latitude' : r._6,
            'Start Station Longitude' : r._7,
            'End Station Latitude' : r._10,
            'End Station Longitude' : r._11,
            'User Type' : user_type_adj,
            'User Gender': user_gender_adj,
            'User Birth Year' : r._14,
            'User Age' : user_age
        }

        cb_dict_list.append( cb_path_item )

        # pprint( cb_path_item )
        # print(" ")

        # Save the current Bike ID
        bikeid_last = r.bikeid

        # Save the current End Station ID
        endstationid_last = end_station_id

        # Increment the path order (for the next iteration)
        path_order += 1

        # DEBUG *******************************************************************************
        # if r.Index > 9: break
        # DEBUG *******************************************************************************

    # Create a DataFrame based upon the list of dictionaries
    cb_paths_df = pd.DataFrame(cb_dict_list)

    return cb_paths_df

In [4]:
# Import CitiBike data
# cb_i_file_list = [ "JC-201902-citibike-tripdata.csv",
#                    "JC-201901-citibike-tripdata.csv",
#                    "201902-citibike-tripdata.csv",
#                    "201901-citibike-tripdata.csv" ]

cb_i_file_list = [ "201901-citibike-tripdata.csv" ]

In [5]:
%%time
# Loop through the list of CitiBike data files to process
for cb_i_file in cb_i_file_list:
    
    print(f"PROCESSING FILE: { cb_i_file }")
    
    # Read the CSV file into a DataFrame
    %time cb_i_df = pd.read_csv(cb_i_file, index_col=False)
    
    # Print a progress message
    print(f"> Records in this file: { cb_i_df['bikeid'].count() }")
    
    # Process path data in the DataFrame
    %time cb_o_df = gen_paths_df( cb_i_df )

    # Print a progress message
    print(f"> Paths in this file: { cb_o_df['Path ID'].nunique() }")

    # Write the processed DataFrame to a CSV file
    cb_o_file = "Paths-" + cb_i_file
    print(f"> Writing the processed data to the output file: { cb_o_file }")    
    %time cb_o_df.to_csv(cb_o_file, index=False)
    
    print(" ")
    

PROCESSING FILE: 201901-citibike-tripdata.csv
Wall time: 4.46 s
> Records in this file: 967287
>>>> Starting a new Path ID: 14529_2019-01-17_17_36_26_2005
>>>> Starting a new Path ID: 14530_2019-01-01_11_25_49_3320
>>>> Starting a new Path ID: 14530_2019-01-19_06_53_52_3383
>>>> Starting a new Path ID: 14530_2019-01-25_15_23_41_3538
>>>> Starting a new Path ID: 14531_2019-01-02_17_54_40_466
>>>> Starting a new Path ID: 14531_2019-01-09_16_39_54_3664
>>>> Starting a new Path ID: 14535_2019-01-01_11_52_57_347
>>>> Starting a new Path ID: 14535_2019-01-03_08_49_17_3295
>>>> Starting a new Path ID: 14536_2019-01-08_13_41_38_366
>>>> Starting a new Path ID: 14537_2019-01-01_14_22_23_3318
>>>> Starting a new Path ID: 14537_2019-01-11_08_30_10_3718
>>>> Starting a new Path ID: 14537_2019-01-13_18_15_33_3349
>>>> Starting a new Path ID: 14539_2019-01-04_16_28_44_3518
>>>> Starting a new Path ID: 14539_2019-01-11_18_56_20_285
>>>> Starting a new Path ID: 14539_2019-01-17_08_15_24_3289
>>>> Star

>>>> Starting a new Path ID: 14622_2019-01-03_14_11_21_3390
>>>> Starting a new Path ID: 14622_2019-01-06_21_25_30_402
>>>> Starting a new Path ID: 14622_2019-01-23_09_25_21_3283
>>>> Starting a new Path ID: 14625_2019-01-01_13_46_55_167
>>>> Starting a new Path ID: 14625_2019-01-15_13_10_29_3126
>>>> Starting a new Path ID: 14625_2019-01-15_13_51_04_3122
>>>> Starting a new Path ID: 14626_2019-01-20_16_43_25_3403
>>>> Starting a new Path ID: 14627_2019-01-01_11_35_14_481
>>>> Starting a new Path ID: 14627_2019-01-03_08_39_42_432
>>>> Starting a new Path ID: 14627_2019-01-09_13_01_18_3539
>>>> Starting a new Path ID: 14627_2019-01-14_09_55_06_487
>>>> Starting a new Path ID: 14627_2019-01-24_09_49_30_3411
>>>> Starting a new Path ID: 14628_2019-01-03_23_12_24_3690
>>>> Starting a new Path ID: 14628_2019-01-07_18_42_19_72
>>>> Starting a new Path ID: 14628_2019-01-11_09_15_09_3320
>>>> Starting a new Path ID: 14628_2019-01-15_08_40_01_545
>>>> Starting a new Path ID: 14628_2019-01-30_07

>>>> Starting a new Path ID: 14723_2019-01-30_08_39_39_3164
>>>> Starting a new Path ID: 14726_2019-01-01_13_21_03_3684
>>>> Starting a new Path ID: 14727_2019-01-02_18_31_46_3551
>>>> Starting a new Path ID: 14728_2019-01-01_09_45_02_3344
>>>> Starting a new Path ID: 14728_2019-01-17_08_28_33_262
>>>> Starting a new Path ID: 14728_2019-01-18_15_58_18_3486
>>>> Starting a new Path ID: 14729_2019-01-01_18_30_33_397
>>>> Starting a new Path ID: 14730_2019-01-01_16_35_22_3363
>>>> Starting a new Path ID: 14730_2019-01-20_11_13_29_503
>>>> Starting a new Path ID: 14730_2019-01-22_09_18_53_3145
>>>> Starting a new Path ID: 14731_2019-01-01_11_43_47_3462
>>>> Starting a new Path ID: 14731_2019-01-01_15_52_14_3660
>>>> Starting a new Path ID: 14732_2019-01-01_13_38_56_453
>>>> Starting a new Path ID: 14732_2019-01-07_11_48_03_244
>>>> Starting a new Path ID: 14732_2019-01-08_20_42_50_3576
>>>> Starting a new Path ID: 14733_2019-01-08_15_33_34_3046
>>>> Starting a new Path ID: 14734_2019-01-01

>>>> Starting a new Path ID: 14833_2019-01-29_15_26_12_411
>>>> Starting a new Path ID: 14838_2019-01-02_10_54_34_274
>>>> Starting a new Path ID: 14838_2019-01-11_18_05_21_239
>>>> Starting a new Path ID: 14838_2019-01-31_19_38_43_281
>>>> Starting a new Path ID: 14839_2019-01-01_16_54_28_238
>>>> Starting a new Path ID: 14839_2019-01-23_09_02_57_445
>>>> Starting a new Path ID: 14841_2019-01-29_11_51_11_3551
>>>> Starting a new Path ID: 14843_2019-01-01_15_37_37_3054
>>>> Starting a new Path ID: 14843_2019-01-03_17_37_43_3223
>>>> Starting a new Path ID: 14843_2019-01-09_09_17_00_3164
>>>> Starting a new Path ID: 14843_2019-01-14_17_11_01_469
>>>> Starting a new Path ID: 14849_2019-01-01_10_51_50_363
>>>> Starting a new Path ID: 14849_2019-01-07_08_53_14_432
>>>> Starting a new Path ID: 14849_2019-01-22_08_11_22_432
>>>> Starting a new Path ID: 14849_2019-01-28_08_35_41_3226
>>>> Starting a new Path ID: 14851_2019-01-02_08_48_26_494
>>>> Starting a new Path ID: 14851_2019-01-30_09_54

>>>> Starting a new Path ID: 14913_2019-01-23_08_43_44_3718
>>>> Starting a new Path ID: 14913_2019-01-23_20_49_29_223
>>>> Starting a new Path ID: 14913_2019-01-27_10_35_13_3162
>>>> Starting a new Path ID: 14913_2019-01-28_13_37_10_465
>>>> Starting a new Path ID: 14914_2019-01-02_13_18_33_3311
>>>> Starting a new Path ID: 14914_2019-01-13_13_01_45_245
>>>> Starting a new Path ID: 14914_2019-01-15_12_40_02_3544
>>>> Starting a new Path ID: 14915_2019-01-02_09_08_41_3303
>>>> Starting a new Path ID: 14916_2019-01-03_16_00_09_3491
>>>> Starting a new Path ID: 14916_2019-01-10_15_17_42_498
>>>> Starting a new Path ID: 14916_2019-01-11_08_53_05_3718
>>>> Starting a new Path ID: 14920_2019-01-02_07_38_44_485
>>>> Starting a new Path ID: 14920_2019-01-15_14_35_09_295
>>>> Starting a new Path ID: 14920_2019-01-23_22_29_26_498
>>>> Starting a new Path ID: 14922_2019-01-09_08_13_14_3389
>>>> Starting a new Path ID: 14922_2019-01-12_16_59_47_314
>>>> Starting a new Path ID: 14922_2019-01-16_09

...

>>>> Starting a new Path ID: 35718_2019-01-06_22_39_13_3398
>>>> Starting a new Path ID: 35718_2019-01-10_07_16_51_3358
>>>> Starting a new Path ID: 35718_2019-01-10_13_05_09_302
>>>> Starting a new Path ID: 35718_2019-01-11_13_43_59_3163
>>>> Starting a new Path ID: 35718_2019-01-12_22_50_24_477
>>>> Starting a new Path ID: 35718_2019-01-22_09_07_27_3584
>>>> Starting a new Path ID: 35719_2019-01-01_11_25_50_472
>>>> Starting a new Path ID: 35719_2019-01-03_18_43_02_499
>>>> Starting a new Path ID: 35719_2019-01-07_08_46_25_486
>>>> Starting a new Path ID: 35719_2019-01-15_13_43_27_442
>>>> Starting a new Path ID: 35719_2019-01-16_08_17_59_309
>>>> Starting a new Path ID: 35719_2019-01-19_03_02_02_3432
>>>> Starting a new Path ID: 35719_2019-01-22_09_18_44_3562
>>>> Starting a new Path ID: 35719_2019-01-31_16_41_45_3463
>>>> Starting a new Path ID: 35720_2019-01-01_07_41_17_3165
>>>> Starting a new Path ID: 35720_2019-01-07_15_59_34_472
>>>> Starting a new Path ID: 35720_2019-01-14_07

>>>> Starting a new Path ID: 35741_2019-01-10_17_13_48_3319
>>>> Starting a new Path ID: 35741_2019-01-12_13_09_11_466
>>>> Starting a new Path ID: 35741_2019-01-13_14_20_29_349
>>>> Starting a new Path ID: 35741_2019-01-18_08_11_28_3354
>>>> Starting a new Path ID: 35741_2019-01-31_13_58_07_3664
>>>> Starting a new Path ID: 35742_2019-01-01_01_56_32_324
>>>> Starting a new Path ID: 35742_2019-01-05_09_30_47_499
>>>> Starting a new Path ID: 35742_2019-01-12_11_41_46_174
>>>> Starting a new Path ID: 35743_2019-01-01_09_46_28_3151
>>>> Starting a new Path ID: 35743_2019-01-03_05_32_24_492
>>>> Starting a new Path ID: 35743_2019-01-06_07_41_10_3590
>>>> Starting a new Path ID: 35743_2019-01-14_17_08_24_484
>>>> Starting a new Path ID: 35743_2019-01-16_08_09_16_519
>>>> Starting a new Path ID: 35743_2019-01-19_15_59_05_3429
>>>> Starting a new Path ID: 35743_2019-01-24_00_44_14_3404
>>>> Starting a new Path ID: 35744_2019-01-02_00_50_58_302
>>>> Starting a new Path ID: 35744_2019-01-02_16_

>>>> Starting a new Path ID: 35764_2019-01-01_09_19_06_422
>>>> Starting a new Path ID: 35764_2019-01-02_12_49_41_493
>>>> Starting a new Path ID: 35764_2019-01-03_09_23_23_3711
>>>> Starting a new Path ID: 35764_2019-01-05_01_20_59_470
>>>> Starting a new Path ID: 35764_2019-01-07_14_01_33_458
>>>> Starting a new Path ID: 35764_2019-01-09_13_44_18_3583
>>>> Starting a new Path ID: 35764_2019-01-13_17_00_32_528
>>>> Starting a new Path ID: 35764_2019-01-20_13_58_51_3321
>>>> Starting a new Path ID: 35764_2019-01-22_07_39_59_382
>>>> Starting a new Path ID: 35764_2019-01-26_12_38_01_265
>>>> Starting a new Path ID: 35764_2019-01-28_08_11_51_445
>>>> Starting a new Path ID: 35765_2019-01-01_08_09_46_472
>>>> Starting a new Path ID: 35765_2019-01-03_00_21_29_436
>>>> Starting a new Path ID: 35765_2019-01-10_07_50_52_3358
>>>> Starting a new Path ID: 35765_2019-01-11_11_07_35_3664
>>>> Starting a new Path ID: 35765_2019-01-13_20_04_25_342
>>>> Starting a new Path ID: 35765_2019-01-14_16_27

>>>> Starting a new Path ID: 35784_2019-01-28_16_22_41_442
>>>> Starting a new Path ID: 35785_2019-01-01_00_18_57_3171
>>>> Starting a new Path ID: 35785_2019-01-01_11_41_24_3164
>>>> Starting a new Path ID: 35785_2019-01-02_17_51_18_517
>>>> Starting a new Path ID: 35785_2019-01-05_23_50_32_3684
>>>> Starting a new Path ID: 35785_2019-01-10_11_10_02_500
>>>> Starting a new Path ID: 35785_2019-01-13_15_51_53_546
>>>> Starting a new Path ID: 35785_2019-01-15_06_38_10_305
>>>> Starting a new Path ID: 35785_2019-01-20_11_32_37_3434
>>>> Starting a new Path ID: 35785_2019-01-20_21_02_26_3155
>>>> Starting a new Path ID: 35785_2019-01-25_19_46_52_3718
>>>> Starting a new Path ID: 35786_2019-01-01_07_46_52_528
>>>> Starting a new Path ID: 35786_2019-01-04_15_50_10_3368
>>>> Starting a new Path ID: 35786_2019-01-05_12_34_40_3042
>>>> Starting a new Path ID: 35786_2019-01-08_07_51_22_285
>>>> Starting a new Path ID: 35786_2019-01-10_12_14_42_519
>>>> Starting a new Path ID: 35786_2019-01-15_13

>>>> Starting a new Path ID: 35804_2019-01-01_06_02_48_3165
>>>> Starting a new Path ID: 35804_2019-01-05_13_21_49_3552
>>>> Starting a new Path ID: 35804_2019-01-05_19_30_23_522
>>>> Starting a new Path ID: 35804_2019-01-08_21_45_38_267
>>>> Starting a new Path ID: 35804_2019-01-31_06_42_47_258
>>>> Starting a new Path ID: 35805_2019-01-01_01_54_15_467
>>>> Starting a new Path ID: 35805_2019-01-15_08_58_26_267
>>>> Starting a new Path ID: 35806_2019-01-01_06_10_19_529
>>>> Starting a new Path ID: 35806_2019-01-02_12_38_22_493
>>>> Starting a new Path ID: 35806_2019-01-06_00_41_45_461
>>>> Starting a new Path ID: 35806_2019-01-17_06_41_21_3641
>>>> Starting a new Path ID: 35806_2019-01-17_07_57_41_472
>>>> Starting a new Path ID: 35806_2019-01-22_18_16_37_3398
>>>> Starting a new Path ID: 35806_2019-01-27_19_30_57_3259
>>>> Starting a new Path ID: 36383_2019-01-06_11_56_05_241
>>>> Starting a new Path ID: 36383_2019-01-18_07_59_45_523
>>>> Starting a new Path ID: 36383_2019-01-31_18_36

In [6]:
cb_o_file

'Paths-201901-citibike-tripdata.csv'